# Dynamic plugins for Backstage: Frontend plugin demo

**Agenda:**

1. View the demo deployment of Backstage
2. Create a new "static" frontend Backstage plugin
3. Update the plugin to be dynamic
4. Build and publish to NPM registry
5. Configure and consume the plugin in Backstage instance

# 🖥️ Review existing Backstage deployment

| OpenShift Console | [https://console-openshift-console.apps-crc.testing/](https://console-openshift-console.apps-crc.testing/) |
| --- | --- |
| **Demo Backstage instance** | **[https://backstage-developer-hub-backstage.apps-crc.testing/](https://backstage-developer-hub-backstage.apps-crc.testing/)** |
| **NPM registry** | **[http://verdaccio-verdaccio.apps-crc.testing/](http://verdaccio-verdaccio.apps-crc.testing/)** |

> Review following preconditions:
> 
> 1. Open the [Backstage instance](https://backstage-developer-hub-backstage.apps-crc.testing/).
> 2. Examine the sidebar menu and recognize that there's no entry "This is Demo" yet. Tell that after we create our plugin, it will appear there.
> 3. Navigate to the Catalog and go to the single Component's overview page. Recount the available cards - there's no card "Entity name" card there. Tell that after we create our plugin, it will appear there.
> 4. Go to the [NPM registry](http://verdaccio-verdaccio.apps-crc.testing/) site and show that there's no packages published in there. Tell that after we publish our plugin, it will appear there.

# 🧑‍💻 Create a new plugin

All steps here are the same as when we are creating a static plugin

> Comment on how we're using the standard Backstage mechanism on creating plugins. That there's nothing unfamiliar or different to what users are already used to at this stage. Plugin is bootstrapped, scaffolded, and ready to be developed as a static plugin.

## 1️⃣ Let's start by bootstrapping a new plugin using `@backstage/cli`

In [11]:
%%sh
cd ../plugins

npx @backstage/cli@0.26.4 new --scope @rhsummit2024 --select plugin --option id=demo


Creating frontend plugin @rhsummit2024/backstage-plugin-demo

 Checking Prerequisites:
  availability  backstage-plugin-demo ✔ 
  creating      temp dir ✔ 

 Executing Template:
  copying       .eslintrc.js ✔ 
  templating    README.md.hbs ✔ 
  copying       tsconfig.json ✔ 
  templating    package.json.hbs ✔ 
  templating    index.tsx.hbs ✔ 
  templating    index.ts.hbs ✔ 
  templating    plugin.test.ts.hbs ✔ 
  templating    routes.ts.hbs ✔ 
  templating    plugin.ts.hbs ✔ 
  copying       setupTests.ts ✔ 
  templating    ExampleComponent.test.tsx.hbs ✔ 
  templating    ExampleComponent.tsx.hbs ✔ 
  copying       index.ts ✔ 
  templating    ExampleFetchComponent.test.tsx.hbs ✔ 
  templating    ExampleFetchComponent.tsx.hbs ✔ 
  copying       index.ts ✔ 

 Installing:
  moving        backstage-plugin-demo ✔ 
  executing     yarn install ✔ 
  executing     yarn lint --fix ✔ 

🎉  Successfully created plugin



## 2️⃣ Next, let's copy over some example files and components

> This step is here so we can show more features packed within a single plugin. All we're doing is copying over some example files and components to the plugin. We'll review them once they are copied over.

In [12]:
%%bash

echo -e "1️⃣\tCopying over example files..."
cp -r ./src/components/DemoEntityCard ../plugins/backstage-plugin-demo/src/components
cp app-config.yaml ../plugins/backstage-plugin-demo/
cp config.d.ts ../plugins/backstage-plugin-demo/

echo "export { demoPlugin, DemoPage } from './plugin';" > ../plugins/backstage-plugin-demo/src/index.ts
echo "export { DemoEntityCard } from './components/DemoEntityCard';" >> ../plugins/backstage-plugin-demo/src/index.ts
echo "export { default as DemoIcon } from '@material-ui/icons/EmojiPeople';" >> ../plugins/backstage-plugin-demo/src/index.ts

echo -e "2️⃣\tInstalling dependencies for the examples above..."
cd ../plugins/backstage-plugin-demo
yarn add -s -D react@^17.0.2 react-dom@^17.0.2 react-router-dom@^6.20.0  2> >(grep -v warning 1>&2)
yarn add -s @backstage/plugin-catalog-react @material-ui/icons@4.11.3  2> >(grep -v warning 1>&2)

echo -e "🎉\tDone"

1️⃣	Copying over example files...
2️⃣	Installing dependencies for the examples above...
🎉	Done


## 3️⃣ Review example files

| Plugin `index.ts` | [./src/index.ts](../plugins/backstage-plugin-demo/src/index.ts)  |
| --- | --- |
| **Example component** | **[./src/components/DemoEntityCard/DemoEntityCard.tsx](../plugins/backstage-plugin-demo/src/components/DemoEntityCard/DemoEntityCard.tsx)** |
| **Plugin config schema** | **[./config.d.ts](../plugins/backstage-plugin-demo/config.d.ts)** |

> Explain the changes we did in the previous step.
> 
> 1. We've added a new component `DemoEntityCard` that will be used to render a card on the entity page.
> 2. We're copied over a `config.d.ts` because we want to use Backstage's configuration mechanism to provide custom configuration values to our plugin and demonstate that a dynamic > plugin can actually consume Backstage APIs and read from config.
> 3. We've installed some dev dependencies - `react`, `react-dom`, etc. There's are normally listed as peer dependencies since Backstage already has them installed. For static plugin > this is enough, since it is usually developed alongside the Backstage app, so the dependencies are already present. For dynamic plugins, we need to install them ourselves to make > them available at build time.
> 4. We've installed some additional dependencies that we want to use to demonstrate additional capabilities of dynamic plugins. `@backstage/plugin-catalog-react` to show a dynamic > plugin can actually use APIs provided by Backstage (in this case the catalog API), and `@material-ui/icons` to show that we can use and re-export from third-party libraries in our > dynamic plugins.
> 5. Let's review the Plugin [index.ts](../plugins/backstage-plugin-demo/src/index.ts) file:
>    ```
>    export { demoPlugin, DemoPage } from './plugin';
>    export { DemoEntityCard } from './components/DemoEntityCard';
>    export { default as DemoIcon } from '@material-ui/icons/EmojiPeople';
>    ```
>    First line comes from the plugin bootstrap - this is what you get when you create a new plugin. No change here.
>    Second line allow us to export a component we created extra, we'll review that component in a moment.
>    Third line shows how we re-export from a third-party library. This way our dynamic plugin provides a new icon that can be used in Backstage.
> 6. Now let's take a look at the [example component](../plugins/backstage-plugin-demo/src/components/DemoEntityCard/DemoEntityCard.tsx) we've added to the plugin. We're interested in commenting on these lines:
>    -  ```
>       export const DemoEntityCard = ({title}: {title: string}) => {
>       ```
>       This line demonstrates that dynamic plugins can pass props to components via config. In this case, we're passing a `title` prop to the card.
>    -  ```
>       const {entity} = useEntity()
>       ```
>       This line shows we can consume internal Backstage API, external to the plugin itself. In this case, we're using the `useEntity` hook to get the entity data from the catalog.
>    -  ```
>       const configApi = useApi(configApiRef);
>       const foo = configApi.getOptionalString('testPlugin.foo');
>       ```
>       These lines show how we can consume the config API to read values from the Backstage configuration. In this case, we're reading a value `testPlugin.foo` from the config. The next file we'll review is the `config.d.ts` file where we define this value.
>    -  Now comment on the component body, how it's going to display the `title` prop as the card header, the entity kind and name on the first line of the card content and the value we're passing via config as a second line of the card content.
> 6. Finally, let's review the [`config.d.ts`](../plugins/backstage-plugin-demo/config.d.ts) file. There you can see a standard Backstage config schema that defines the `testPlugin.foo` value we're reading in the component. This is how we can provide custom configuration values to our dynamic plugin.

## 4️⃣ Prepare package for release

> Again, this is standard procedure. Nothing here is special to dynamic plugins. We set the NPM package as publishable (by deleting the `private` key from `package.json`) and set a `configSchema` property - this is a well-known Backstage specific property expected in the `package.json` file of a plugin. It's used to define the configuration schema for the plugin.

In [13]:
%%sh
cd ../plugins/backstage-plugin-demo

echo "1️⃣\tExport plugin config schema..."
npm pkg set "configSchema"="config.d.ts"

echo "2️⃣\tMark package as public.."
npm pkg delete "private"

echo "🎉\tDone"

1️⃣	Export plugin config schema...
2️⃣	Mark package as public..
🎉	Done


# 🙇 Make plugin dynamic

🎉🎉🎉🎉 **This is the new stuff!** 🎉🎉🎉🎉

This follows steps from [Janus Showcase Dynamic plugins docs](https://github.com/janus-idp/backstage-showcase/blob/main/showcase-docs/dynamic-plugins.md#frontend-plugins).

> Comment on how we're now making the plugin dynamic. This is the new stuff that we're showing here. We're going to make the plugin dynamic by installing the `@janus-idp/cli` dependency that will help us build the browser assets as a build artifact. We're also listing these assets which are stored in `dist-scalprum` folder to be included in the final NPM package. In addition to that we're setting some extra `script` entries to make the build process easier.

In [14]:
%%bash
cd ../plugins/backstage-plugin-demo

echo -e "1️⃣\tInstall @janus-idp/cli..."
yarn add -s -D @janus-idp/cli  2> >(grep -v warning 1>&2)

echo -e "2️⃣\tAdd exported assets to the package bundle..."
npm pkg set "files[1]"="dist-scalprum"

echo -e "3️⃣\tAdd package scripts for release automation..."
npm pkg set "scripts.export-dynamic"="janus-cli package export-dynamic-plugin"
npm pkg set "scripts.postversion"="yarn run export-dynamic"

echo -e "🎉\tDone"

1️⃣	Install @janus-idp/cli...
2️⃣	Add exported assets to the package bundle...
3️⃣	Add package scripts for release automation...
🎉	Done


# 📦 Build and publish

> This is the same as for static plugins, but we're building the plugin as a dynamic plugin thanks to the additional `yarn export-dynamic` step. Once the plugin is built, we're publishing it to the NPM registry.

In [15]:
%%sh
cd ../plugins/backstage-plugin-demo

yarn tsc
yarn build
yarn export-dynamic
yarn publish

yarn run v1.22.22
$ /Users/tcoufal/Workspace/demo/plugins/backstage-plugin-demo/node_modules/.bin/tsc
Done in 2.60s.
yarn run v1.22.22
$ backstage-cli package build


(node:41097) [DEP0040] DeprecationWarning: The `punycode` module is deprecated. Please use a userland alternative instead.
(Use `node --trace-deprecation ...` to show where the warning was created)


Done in 1.44s.
yarn run v1.22.22
$ janus-cli package export-dynamic-plugin


(node:41106) [DEP0040] DeprecationWarning: The `punycode` module is deprecated. Please use a userland alternative instead.
(Use `node --trace-deprecation ...` to show where the warning was created)
No scalprum config. Using default dynamic UI configuration:
{
  "name": "rhsummit2024.backstage-plugin-demo",
  "exposedModules": {
    "PluginRoot": "./src/index.ts"
  }
}
If you wish to change the defaults, add "scalprum" configuration to plugin "package.json" file, or use the '--scalprum-config' option to specify an external config.
Generating dynamic frontend plugin assets in /Users/tcoufal/Workspace/demo/plugins/backstage-plugin-demo/dist-scalprum


  354.96 kB  dist-scalprum/static/758.1f7bbe3b.chunk.js
  40.18 kB   dist-scalprum/static/961.eceed8cd.chunk.js
  32.5 kB    dist-scalprum/static/react-syntax-highlighter_languages_highlight_mathematica.6de3b12e.chunk.js
  18.52 kB   dist-scalprum/static/305.3fbfb654.chunk.js
  16.73 kB   dist-scalprum/static/react-syntax-highlighter_languages_highlight_isbl.ed19a356.chunk.js
  16.09 kB   dist-scalprum/static/5588.01099cb9.chunk.js
  14.26 kB   dist-scalprum/static/9333.18995370.chunk.js
  13.27 kB   dist-scalprum/static/react-syntax-highlighter_languages_highlight_gml.205c2e82.chunk.js
  10.88 kB   dist-scalprum/static/react-syntax-highlighter_languages_highlight_maxima.e484f1fd.chunk.js
  10.75 kB   dist-scalprum/static/react-syntax-highlighter_languages_highlight_sqf.a1d41787.chunk.js
  10.59 kB   dist-scalprum/static/react-syntax-highlighter_languages_highlight_oneC.195fcaf2.chunk.js
  10.4 kB    dist-scalprum/static/9277.d445c9eb.chunk.js
  9.29 kB    dist-scalprum/static/react-sy

Saving self-contained config schema in /Users/tcoufal/Workspace/demo/plugins/backstage-plugin-demo/dist-scalprum/configSchema.json


Done in 17.25s.
yarn publish v1.22.22
[1/4] Bumping version...
info Current version: 0.1.0
[2/4] Logging in...
[3/4] Publishing...
$ backstage-cli package prepack
$ backstage-cli package postpack
success Published.
[4/4] Revoking token...
info Not revoking login token, specified via config file.
Done in 1.50s.


Review published package `@rhsummit2024/backstage-plugin-demo`

> Comment on how the package is locate in our local Verdaccio NPM registry, that it indeed was published a few moments ago. You can also view the package listed on the [Verdaccio site](http://verdaccio-verdaccio.apps-crc.testing/).

In [16]:
!npm info @rhsummit2024/backstage-plugin-demo


@rhsummit2024/backstage-plugin-demo@0.1.0 | Apache-2.0 | deps: 8 | versions: 1
Welcome to the demo plugin!

dist
.tarball: http://verdaccio-verdaccio.apps-crc.testing/@rhsummit2024/backstage-plugin-demo/-/backstage-plugin-demo-0.1.0.tgz
.shasum: d07460f2325ff51ee2cc937131808a515bcba71c
.integrity: sha512-ttIGu9pgI5aC6VxSa2VF5oI2oAHXUKRrs/qeTXsRAo5ej3HfArjtKxMDV1GU6voatlvA2Vz3QqZnN4vplIuZEA==

dependencies:
@backstage/core-components: ^0.14.4
@backstage/core-plugin-api: ^1.9.2
@backstage/plugin-catalog-react: ^1.11.3
@backstage/theme: ^0.5.3
@material-ui/core: ^4.12.2
@material-ui/icons: 4.11.3
@material-ui/lab: 4.0.0-alpha.61
react-use: ^17.2.4

dist-tags:
latest: 0.1.0  

published just now


# ☁️ Deploy

Now let's update our Backstage instance with the new Plugin

| Helm values snippet | [values.yaml](./values.yaml)  |
| --- | --- |

> Comment on the values.yaml file we're applying to our Backstage instance:
> 1. `- package: '@rhsummit2024/backstage-plugin-demo@0.1.0'` - this is the new plugin we've just published.
> 2. `integrity: "" # FIXME: CHANGE ME!` - Tell that this is inferred by `npm info` below
> 3. `appIcons:` section - Here we're instructing Backstage to use a `DemoIcon` exported symbol and store it in Backstage icon registry as `demoIcon`
> 4. `dynamicRoutes:` section - Here we're instructing Backstage to use the `DemoPage` exported symbol and expose it as an Backstage UI route on `/demo` path. In addition to that, we're instructing Backstage to list this new page in the sidebar menu as "This is Demo" entry with `demoIcon` icon (this is the same icon we defined in the previous step).
> 5. `mountPoints:` section - Here we're configuring Backstage to mount the `DemoEntityCard` component to `entity.page.overview/cards` catalog entity Overview tab as a card. We pass it a custom `title` prop and provide a layout constraint for card placement. In addition we provide a `if` condition that will only render the card if the entity kind is `Component`.
>
> You can comment on that all possible configuration options are listed in the  [Janus Showcase Dynamic plugins docs](https://github.com/janus-idp/backstage-showcase/blob/main/showcase-docs/dynamic-plugins.md#frontend-plugins).

In [17]:
%%bash

INTEGRITY=$(npm info @rhsummit2024/backstage-plugin-demo --json | jq -r '.dist.integrity')
helm upgrade  -i backstage --reuse-values -f values.yaml --set global.dynamic.plugins[0].integrity=$INTEGRITY ../00_setup/developer-hub-1.1.0.tgz

Release "backstage" has been upgraded. Happy Helming!
NAME: backstage
LAST DEPLOYED: Wed May  8 14:25:58 2024
NAMESPACE: backstage
STATUS: deployed
REVISION: 2


> Go to the OpenShift console and observe new Backstage pod rollout - This is best obseved from the Developer perspective topology view, where you can see a new pod being scheduled and then you can jump into the new pods logs. Show the initContainer logs to show that the new plugin is being downloaded and installed. Once the pod is running, go back to the topology view and wait until it reports ready and the old pod is removed.
> 
> Then navigate to the Backstage instance and show the new "This is Demo" entry in the sidebar menu. Navigate to the Catalog and show the new "Entity name" card on the Component's overview page.

# 🧹 Optional: Cleanup

Let's undeploy and unpublish the plugin

In [ ]:
%%bash


helm upgrade  -i backstage --reuse-values -f values.yaml --set global.dynamic.plugins=null ../00_setup/developer-hub-1.1.0.tgz
npm unpublish --force @rhsummit2024/backstage-plugin-demo || true
rm -rf ../plugins/backstage-plugin-demo